# 0.矩阵及其运算
## 0.1.环境、库

In [1]:
%matplotlib inline
import numpy as np
import sympy as sp
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from functools import reduce
plt.rcParams['font.sans-serif'] = ['SimHei'] #指定默认字体   
plt.rcParams['axes.unicode_minus'] = False #解决保存图像是负号'-'显示为方块的问题
x, y, z, t = sp.symbols('x y z t')
k, m, n = sp.symbols('k m n', integer=True)
f, g, h = sp.symbols('f g h', cls=sp.Function)
sp.init_printing()
# common tool function
def getM(var_name,r,c):
    """
    make a sympy s matrix
    :param var_name:the symbol name
    :param r:matrix s rows
    :param c:matrix s cols
    """
    if r>1 and c>1:
        ss=sp.symbols("{0}_{{{{0:{1}}}{{0:{2}}}}}".format(var_name,r,c))        
    elif r==1:
        ss=sp.symbols("{0}_{{0:{1}}}".format(var_name,c))
    elif c==1:
        ss=sp.symbols("{0}_{{0:{1}}}".format(var_name,r))
    else:
        raise Exception('invalid input parameter！')
    return sp.Matrix([ss[i*c:(i+1)*c] for i in range(r)])

In [2]:
%%HTML
<style>
table {width:95%;}
table td, table th, table tr {text-align:left !important;border: 1px  #888888 solid !important;}
</style>

# 1.矩阵
内容提要
>1. 矩阵的定义
2. 特殊的矩阵
3. 矩阵与线性变换

## 1.1.定义
数学上，一个$m*n$的矩阵是一个m行n列元素排列形成的矩形阵列，矩阵里的元素可以是数字、符号或者数学式。在图像处理、人工智能、量化投资等领域，使用矩阵来表示和处理数据非常常见。矩阵最早来自于方程组的系数及常数所构成的方阵。例如，如下方程组:
> $\begin{cases}
 & 4x_1-5x_2=-13 \\ 
 &-2x_1+3x_2=9 
\end{cases}$

这里有两个方程和两个变量，运用高中代数的知识，我们知道，可以为$x_1和x_2$找到一组唯一的解 (除非方程可以进一步简化，例如，如果第二个方程只是第一个方程的倍数形式。但是显然上面的例子不可简化，是有唯一解的)。在矩阵表达中，我们可以简洁的写作:
>$Ax=b$

其中:
>$A=\begin{bmatrix}4&-5\\-2&3\end{bmatrix},b=\begin{bmatrix}-13\\9\end{bmatrix}$

后面我们会看到，把方程表示成这种形式，在分析线性方程方面有很多优势(包括明显地节省空间)。

**基本符号**
>* $A\in\mathbb{R}^{m\times n}$表示一个$m行n列$的矩阵，并且矩阵$A$中的所有元素都是实数，此时我们称$A$矩阵为实矩阵，如果元素是复数的矩阵，称为复矩阵;
* $X\in\mathbb{R}^n$表示一个含有$n$个元素的向量。通常，我们把$n$维向量看成是一个$n行1列$矩阵，即列向量。如果我们想表示一个行向量（$1行n列$矩阵），我们通常写作$x^T$ ($x^T$表示$x$的转置，后面会解释它的定义);
* 一个向量$X$的第i个元素表示为$x_i$:$X=\begin{bmatrix}x_1\\x_2\\\vdots\\x_n\end{bmatrix}$;
* $a_{ij}或者a_{i,j}$表示矩阵$A$的$第i行j列$的元素:$A=\begin{bmatrix}a_{11}&\dots&a_{1n}\\\vdots&\ddots&\vdots\\a_{m1}&\dots&a_{mn}\end{bmatrix}$;
* $a_{:,j}$表示$A$矩阵的第$j$列元素:$A=\begin{bmatrix}a_{:,1}&\dots&a_{:,n}\end{bmatrix}$;
* $a_{i,:}$表示$A$矩阵的第$i$行元素:$A=\begin{bmatrix}a_{1,:}\\\vdots\\a_{m,:}\end{bmatrix}$

基于上面的定义，矩阵$A$，简记为：$A=A_{m\times n}={(a_{ij})}_{m \times n}=(a_{ij})$

**行列式和矩阵的区别**

|行列式|矩阵|
| --- | --- |
|$|A|$|$A$|
|$\begin{vmatrix}a_{11}&\dots&a_{1n}\\\vdots&\ddots&\vdots\\a_{n1}&\dots&a_{nn}\end{vmatrix}$|$\begin{bmatrix}a_{11}&\dots&a_{1n}\\\vdots&\ddots&\vdots\\a_{m1}&\dots&a_{mn}\end{bmatrix}$|
|<li>行数等于列数;</li><li>共有$n^2$个元素;</li>|<li>行数可以不等于列数;</li><li>共有$m\times n$个元素;</li><li>本质上就是一个数表;</li>|
|$det(a_{ij})$|${(a_{ij})}_{m\times n}$|

## 1.2.特殊的矩阵
1. 行数和列数都等于$n$的矩阵，称为**$n$阶方阵**,可记作$A_n$.
2. 只有一行的矩阵$A=(a_1,\dots,a_n)$称为**行矩阵**(或**行向量**).只有一列的矩阵$B=\begin{pmatrix}a_1\\\vdots\\a_n\end{pmatrix}$称为**列矩阵**(或**列向量**).
3. 元素全为零的矩阵称为**零矩阵**.可记作$O$.
4. 除对角线外全为零的方阵称为**对角阵**.记作$A=diag(\lambda_1,\dots,\lambda_n)$.特别的,对角线全为1的对角阵称为**单位阵**.记作$E_n$.

**同型矩阵与矩阵相等的概念**
1. 两个矩阵的行数相等、列数相等时,称为**同型矩阵**.
2. 两个矩阵$A=(a_{ij})与B=(b_{ij})$为同型矩阵,并且对应元素相等,即$a_{ij}=b_{ij}$,则称矩阵$A与B$相等,记作$A=B$.**注意**：不同型的零矩阵是不相等的.

## 1.3.矩阵与线性变换
$n个变量x_1,\dots,x_n与m个变量y_1,\dots,y_m$之间的关系式:<br>
$\begin{cases}y_1=a_{11}x_1+\dots+a_{1n}x_n\\\vdots\\y_m=a_{m1}x_1+\dots+a_{mn}x_n \end{cases}$

In [3]:
xn=3
xa,xx,xy=getM('a',xn,xn),getM('x',xn,1),getM('y',xn,1)
sp.Eq(xy,xa*xx)

⎡y_{0}⎤   ⎡a_{{0}{0}}⋅x_{0} + a_{{0}{1}}⋅x_{1} + a_{{0}{2}}⋅x_{2}⎤
⎢     ⎥   ⎢                                                      ⎥
⎢y_{1}⎥ = ⎢a_{{1}{0}}⋅x_{0} + a_{{1}{1}}⋅x_{1} + a_{{1}{2}}⋅x_{2}⎥
⎢     ⎥   ⎢                                                      ⎥
⎣y_{2}⎦   ⎣a_{{2}{0}}⋅x_{0} + a_{{2}{1}}⋅x_{1} + a_{{2}{2}}⋅x_{2}⎦

表示一个从变量$x_1,\dots,x_n到变量y_1,\dots,y_m$的线性变换,其中$a_{ij}$为常数.

从上面的关系式中，能够得到系数矩阵：<br>
$A=\begin{bmatrix}a_{11}&\dots&a_{1n}\\\vdots&\ddots&\vdots\\a_{m1}&\dots&a_{mn}\end{bmatrix}$<br>
能够看到线性变换与矩阵之间存在着一一对应关系.

In [4]:
xa

⎡a_{{0}{0}}  a_{{0}{1}}  a_{{0}{2}}⎤
⎢                                  ⎥
⎢a_{{1}{0}}  a_{{1}{1}}  a_{{1}{2}}⎥
⎢                                  ⎥
⎣a_{{2}{0}}  a_{{2}{1}}  a_{{2}{2}}⎦

**恒等变换**，系数矩阵$A$为单位矩阵$E_n$

In [5]:
xa=sp.eye(xn)
xa,sp.Eq(xy,xa*xx)

⎛⎡1  0  0⎤  ⎡y_{0}⎤   ⎡x_{0}⎤⎞
⎜⎢       ⎥  ⎢     ⎥   ⎢     ⎥⎟
⎜⎢0  1  0⎥, ⎢y_{1}⎥ = ⎢x_{1}⎥⎟
⎜⎢       ⎥  ⎢     ⎥   ⎢     ⎥⎟
⎝⎣0  0  1⎦  ⎣y_{2}⎦   ⎣x_{2}⎦⎠

# 2.矩阵的运算
## 2.1.加法
矩阵加法的定义非常复合直觉认识，和算术加法相似。定义：两个$m\times n$矩阵$A=(a_{ij}),B=(b_{ij})$,那么矩阵$A与B$的和记作$A+B$，规定如下：<br>$A+B=\begin{bmatrix}a_{11}+b_{11}&\dots&a_{1n}+b_{1n}\\\vdots&\ddots&\vdots\\a_{m1}+b_{m1}&\dots&a_{mn}+b_{mn}\end{bmatrix}$<br>
**注意**：只有两个矩阵是同型矩阵时，才能进行加法运算.

矩阵加法运算的规律：
<table>
    <tr><th></th><th>$\forall a,b,c \in \mathbb R$</th><th>$A、B、C是同型矩阵$</th></tr>
    <tr><td>交换律</td><td>$a+b=b+a$</td><td>$A+B=B+A$</td></tr>
    <tr><td>结合律</td><td>$(a+b)+c=a+(b+c)$</td><td>$(A+B)+C=A+(B+C)$</td></tr>
    <tr><td>其他</td><td colspan=2>设矩阵$A=(a_{ij}),记-A=(-a_{ij})$,称为矩阵$A$的**负矩阵**.显然$A+(-A)=0,A-B=A+(-B)$</td></tr>
<table>

In [6]:
xa,xb,xc=[getM(i,xn,xn) for i in 'abc']
xa+xb,xb+xa

⎛⎡a_{{0}{0}} + b_{{0}{0}}  a_{{0}{1}} + b_{{0}{1}}  a_{{0}{2}} + b_{{0}{2}}⎤  
⎜⎢                                                                         ⎥  
⎜⎢a_{{1}{0}} + b_{{1}{0}}  a_{{1}{1}} + b_{{1}{1}}  a_{{1}{2}} + b_{{1}{2}}⎥, 
⎜⎢                                                                         ⎥  
⎝⎣a_{{2}{0}} + b_{{2}{0}}  a_{{2}{1}} + b_{{2}{1}}  a_{{2}{2}} + b_{{2}{2}}⎦  

⎡a_{{0}{0}} + b_{{0}{0}}  a_{{0}{1}} + b_{{0}{1}}  a_{{0}{2}} + b_{{0}{2}}⎤⎞
⎢                                                                         ⎥⎟
⎢a_{{1}{0}} + b_{{1}{0}}  a_{{1}{1}} + b_{{1}{1}}  a_{{1}{2}} + b_{{1}{2}}⎥⎟
⎢                                                                         ⎥⎟
⎣a_{{2}{0}} + b_{{2}{0}}  a_{{2}{1}} + b_{{2}{1}}  a_{{2}{2}} + b_{{2}{2}}⎦⎠

In [7]:
xa+xb==xb+xa

True

In [8]:
(xa+xb)+xc==xa+(xb+xc)

True

## 2.2.减法
矩阵减法也和加法一样简单。$A_{i,j}-B_{i,j}=\begin{bmatrix}a_{11}-b_{11}&\dots&a_{1n}-b_{1n}\\\vdots&\ddots&\vdots\\a_{m1}-b_{m1}&\dots&a_{mn}-b_{mn}\end{bmatrix}$。对于上面给出的矩阵，有：

In [9]:
xa-xb==xa+(-xb)

True

## 2.3.数与矩阵相乘
**定义**：数$\lambda$与矩阵$A$的乘积记作**$\lambda A$或者$A\lambda$**,如下：

In [10]:
xa,xa*x,x*xa

⎛⎡a_{{0}{0}}  a_{{0}{1}}  a_{{0}{2}}⎤  ⎡a_{{0}{0}}⋅x  a_{{0}{1}}⋅x  a_{{0}{2}}
⎜⎢                                  ⎥  ⎢                                      
⎜⎢a_{{1}{0}}  a_{{1}{1}}  a_{{1}{2}}⎥, ⎢a_{{1}{0}}⋅x  a_{{1}{1}}⋅x  a_{{1}{2}}
⎜⎢                                  ⎥  ⎢                                      
⎝⎣a_{{2}{0}}  a_{{2}{1}}  a_{{2}{2}}⎦  ⎣a_{{2}{0}}⋅x  a_{{2}{1}}⋅x  a_{{2}{2}}

⋅x⎤  ⎡a_{{0}{0}}⋅x  a_{{0}{1}}⋅x  a_{{0}{2}}⋅x⎤⎞
  ⎥  ⎢                                        ⎥⎟
⋅x⎥, ⎢a_{{1}{0}}⋅x  a_{{1}{1}}⋅x  a_{{1}{2}}⋅x⎥⎟
  ⎥  ⎢                                        ⎥⎟
⋅x⎦  ⎣a_{{2}{0}}⋅x  a_{{2}{1}}⋅x  a_{{2}{2}}⋅x⎦⎠

In [11]:
xa*x==x*xa

True

数乘矩阵的运算规律：
<table>
    <tr><th></th><th>$\forall a,b,c \in \mathbb R$</th><th>$A、B是同型矩阵,\lambda,\mu 是数$</th></tr>
    <tr><td>结合律</td><td>$abc=a(bc)$</td><td>$(\lambda\mu)A=\lambda(\mu A)$</td></tr>
    <tr>
        <td>分配律</td><td>$(a+b)\cdot c=ac+bc$<br>$c \cdot (a+b)=ca+cb$</td>
        <td>$(\lambda+\mu)A=\lambda A+\mu A$<br>$\lambda(A+B)=\lambda A+\lambda B$</td>
    </tr>
    <tr><td>其他</td><td colspan=2>矩阵相加与相乘结合起来，统称为**矩阵的线性运算**</td></tr>
<table>

In [12]:
x*y*xa==x*(y*xa)

True

In [13]:
sp.simplify((x+y)*xa-(x*xa+y*xa))

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

In [14]:
sp.simplify(x*(xa+xb)-x*xa-x*xb)

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

## 2.4.矩阵与矩阵相乘
**定义：**$A_{i,j}$矩阵的每一行的元素分别与$B_{j,k}$矩阵的每一列的元素两两相乘并相加，从而得到一个新的矩阵$C_{i,k}$。两个矩阵能相乘的充分必要条件是第一个矩阵的列数与第二个矩阵的行数相等，否则无定义。$C_{i,k}=A_{i,j}\times B_{j,k},c_{i,j}=\sum_{n=0}^{k-1}(a_{i,n}\times b_{n,j})$。例如：

In [15]:
xa,xb,xc

⎛⎡a_{{0}{0}}  a_{{0}{1}}  a_{{0}{2}}⎤  ⎡b_{{0}{0}}  b_{{0}{1}}  b_{{0}{2}}⎤  ⎡
⎜⎢                                  ⎥  ⎢                                  ⎥  ⎢
⎜⎢a_{{1}{0}}  a_{{1}{1}}  a_{{1}{2}}⎥, ⎢b_{{1}{0}}  b_{{1}{1}}  b_{{1}{2}}⎥, ⎢
⎜⎢                                  ⎥  ⎢                                  ⎥  ⎢
⎝⎣a_{{2}{0}}  a_{{2}{1}}  a_{{2}{2}}⎦  ⎣b_{{2}{0}}  b_{{2}{1}}  b_{{2}{2}}⎦  ⎣

c_{{0}{0}}  c_{{0}{1}}  c_{{0}{2}}⎤⎞
                                  ⎥⎟
c_{{1}{0}}  c_{{1}{1}}  c_{{1}{2}}⎥⎟
                                  ⎥⎟
c_{{2}{0}}  c_{{2}{1}}  c_{{2}{2}}⎦⎠

In [16]:
xa*xb

⎡a_{{0}{0}}⋅b_{{0}{0}} + a_{{0}{1}}⋅b_{{1}{0}} + a_{{0}{2}}⋅b_{{2}{0}}  a_{{0}
⎢                                                                             
⎢a_{{1}{0}}⋅b_{{0}{0}} + a_{{1}{1}}⋅b_{{1}{0}} + a_{{1}{2}}⋅b_{{2}{0}}  a_{{1}
⎢                                                                             
⎣a_{{2}{0}}⋅b_{{0}{0}} + a_{{2}{1}}⋅b_{{1}{0}} + a_{{2}{2}}⋅b_{{2}{0}}  a_{{2}

{0}}⋅b_{{0}{1}} + a_{{0}{1}}⋅b_{{1}{1}} + a_{{0}{2}}⋅b_{{2}{1}}  a_{{0}{0}}⋅b_
                                                                              
{0}}⋅b_{{0}{1}} + a_{{1}{1}}⋅b_{{1}{1}} + a_{{1}{2}}⋅b_{{2}{1}}  a_{{1}{0}}⋅b_
                                                                              
{0}}⋅b_{{0}{1}} + a_{{2}{1}}⋅b_{{1}{1}} + a_{{2}{2}}⋅b_{{2}{1}}  a_{{2}{0}}⋅b_

{{0}{2}} + a_{{0}{1}}⋅b_{{1}{2}} + a_{{0}{2}}⋅b_{{2}{2}}⎤
                                                        ⎥
{{0}{2}} + a_{{1}{1}}⋅b_{{1}{2}} + a_{{1}{2}}⋅b_{{2}{2}}⎥
                                  

矩阵的乘积记作：$C=AB$

一般情况下，矩阵乘法不满足交换律

In [17]:
xa*xb==xb*xa

False

另外，$A\not=O,B\not=O,可能出现AB=O$,而$AB=O,不一定A=O或者B=O$.

**矩阵乘法的运算规律**
1. 乘法结合律

In [18]:
sp.simplify(xa*xb*xc-xa*(xb*xc))

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

2. 数乘和乘法的结合律

In [19]:
sp.simplify(x*xa*xb-x*(xa*xb))

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

3. 乘法对加法的的分配率

In [20]:
sp.simplify(xa*(xb+xc)-(xa*xb+xa*xc)),sp.simplify(xa*(xb+xc)-(xa*xb+xa*xc))

⎛⎡0  0  0⎤  ⎡0  0  0⎤⎞
⎜⎢       ⎥  ⎢       ⎥⎟
⎜⎢0  0  0⎥, ⎢0  0  0⎥⎟
⎜⎢       ⎥  ⎢       ⎥⎟
⎝⎣0  0  0⎦  ⎣0  0  0⎦⎠

4. 单位矩阵在矩阵乘法中的作用类似于数1

In [21]:
xa==xa*sp.eye(xn)

True

5. 矩阵的幂

若$A$是$n$阶方阵,定义$A^k=\overset{\underbrace{A\dots A}}{k}$,显然,$A^kA^l=A^{k+l},{(A^k)}^l=A^{lk}$

In [22]:
sp.simplify(xa**2*xa**3-xa**(2+3))

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

In [23]:
sp.expand((xa**2)**3-xa**(2*3))

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

## 2.5.矩阵的转置
**定义：**把矩阵$A$的行换成同序数的列得到的新矩阵，叫做矩阵$A$的**转置矩阵**,记作$A^T$.

In [24]:
xa,xb=[getM(i,3,2) for i in 'ab']
xa,xa.T

⎛⎡a_{{0}{0}}  a_{{0}{1}}⎤                                      ⎞
⎜⎢                      ⎥  ⎡a_{{0}{0}}  a_{{1}{0}}  a_{{2}{0}}⎤⎟
⎜⎢a_{{1}{0}}  a_{{1}{1}}⎥, ⎢                                  ⎥⎟
⎜⎢                      ⎥  ⎣a_{{0}{1}}  a_{{1}{1}}  a_{{2}{1}}⎦⎟
⎝⎣a_{{2}{0}}  a_{{2}{1}}⎦                                      ⎠

转置矩阵的运算性质:

In [25]:
xa==xa.T.T

True

In [26]:
(xa+xb).T==xa.T+xb.T

True

In [27]:
(x*xa).T==x*xa.T

True

In [28]:
(xa*xb.T).T==xb.T.T*xa.T

True

**定义：**$A$为$n$阶方阵，如果满足$A=A^T$,那么矩阵$A$称为**对称阵**.如果满足$A=-A^T$,那么矩阵$A$称为**反对称阵**.

In [29]:
xa=getM('a',xn,xn)
xa

⎡a_{{0}{0}}  a_{{0}{1}}  a_{{0}{2}}⎤
⎢                                  ⎥
⎢a_{{1}{0}}  a_{{1}{1}}  a_{{1}{2}}⎥
⎢                                  ⎥
⎣a_{{2}{0}}  a_{{2}{1}}  a_{{2}{2}}⎦

In [30]:
sp.solve(xa-xa.T,xa)

In [31]:
sp.solve(xa+xa.T,xa)

## 2.6.方阵的行列式
**定义:**由$n$阶方阵的元素所构成的行列式，叫做**方阵$A$的行列式**,记作$|A|或det(A)$.

**运算性质**:

In [32]:
xn=3
xa,xb=[getM(i,xn,xn) for i in 'ab']

In [33]:
# 1.方阵的行列式与方阵的转置的行列式相等
xa.T.det()==xa.det()

True

In [34]:
# 2.数乘矩阵的行列式与数的阶次幂、乘矩阵的行列式相等
sp.simplify((x*xa).det()-x**xn*xa.det())

In [35]:
# 3.矩阵相乘之后的行列式与矩阵行列式的乘积相等
sp.simplify((xa*xb).det()-xa.det()*xb.det())

In [36]:
# 4.(A与B的乘积)的行列式与(B与A的乘积)的行列式相等
sp.simplify((xa*xb).det()-(xb*xa).det())

**定义:**行列式$|A|$的各个元素的代数余子式$A_{ij}$所构成的如下矩阵：<br>
$A^*=\begin{bmatrix}A_{11}&\dots&A_{n1}\\\vdots&\ddots&\vdots\\A_{1n}&\dots&A_{nn}\end{bmatrix}$
称为矩阵$A$的伴随矩阵.其中元素$a_{ij}$的代数余子式$A_{ij}$位于第**$j行i列$(注意)**.

**性质:**$AA^*=A^*A=|A|E$.(通过行列式性质容易证明)

In [37]:
xa,xa.adjugate()

⎛⎡a_{{0}{0}}  a_{{0}{1}}  a_{{0}{2}}⎤  ⎡a_{{1}{1}}⋅a_{{2}{2}} - a_{{1}{2}}⋅a_{
⎜⎢                                  ⎥  ⎢                                      
⎜⎢a_{{1}{0}}  a_{{1}{1}}  a_{{1}{2}}⎥, ⎢-a_{{1}{0}}⋅a_{{2}{2}} + a_{{1}{2}}⋅a_
⎜⎢                                  ⎥  ⎢                                      
⎝⎣a_{{2}{0}}  a_{{2}{1}}  a_{{2}{2}}⎦  ⎣a_{{1}{0}}⋅a_{{2}{1}} - a_{{1}{1}}⋅a_{

{2}{1}}   -a_{{0}{1}}⋅a_{{2}{2}} + a_{{0}{2}}⋅a_{{2}{1}}  a_{{0}{1}}⋅a_{{1}{2}
                                                                              
{{2}{0}}  a_{{0}{0}}⋅a_{{2}{2}} - a_{{0}{2}}⋅a_{{2}{0}}   -a_{{0}{0}}⋅a_{{1}{2
                                                                              
{2}{0}}   -a_{{0}{0}}⋅a_{{2}{1}} + a_{{0}{1}}⋅a_{{2}{0}}  a_{{0}{0}}⋅a_{{1}{1}

} - a_{{0}{2}}⋅a_{{1}{1}} ⎤⎞
                          ⎥⎟
}} + a_{{0}{2}}⋅a_{{1}{0}}⎥⎟
                          ⎥⎟
} - a_{{0}{1}}⋅a_{{1}{0}} ⎦⎠

In [38]:
sp.simplify(xa*xa.adjugate()-xa.adjugate()*xa)

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

In [39]:
sp.simplify(xa*xa.adjugate()-xa.det()*sp.eye(xn))

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

## 2.7.共轭矩阵
当$A=(a_{ij})$为复矩阵时,用$\overline{a_{ij}}$表示$a_{ij}$的共轭复数,记$\overline{A}=(\overline{a_{ij}})$,$\overline{A}称为A$的共轭矩阵.

**运算性质**(假设A、B为复矩阵,$\lambda$为复数,且运算都是可行的):
1. $\overline{(A+B)}=\overline A+\overline B$
2. $\overline{\lambda A}=\bar \lambda \bar A$
3. $\overline{AB}=\bar A \bar B$.

# 3.逆矩阵
矩阵并没有一个直接叫除法的操作。但有个与之相似的运算，叫做求逆运算。<br>
对于$n$阶单位矩阵**$E$**以及同阶的方阵$A$,有$AE=EA=A$,从乘法的角度来看，$n$阶单位矩阵$E$在同阶方阵中的地位类似于1在复数中的地位.一个复数$a\not =0$的倒数$a^{-1}$可以用等式$aa^{-1}=1$来刻画,类似的，**定义**:矩阵$A$的逆$A^{-1}$被定义为一个与$A$相乘后能得到一个单元矩阵的矩阵。即：$A\times A^{-1}=E$。求逆这个操作本身是可逆的，一个矩阵的逆的逆也是这个矩阵本身。因此$A^{-1}\times A=E$。根据这个特点我们可以推断出能求逆的矩阵，其行数和列数也必然相同。<br>

在方阵行列式一节，我们得到如下结论:$AA^*=A^*A=|A|E$.结合逆矩阵的性质:$A^{-1}\times A=E$.$\to$<br>
**定理:**若$|A|\not =0$,则方阵$A$可逆，而且$A^{-1}=\frac{1}{|A|}A^*$.<br>
**推论:**若$|A|\not =0$,则$|A^{-1}|=\frac{1}{|A|}$.

In [40]:
xn=2
xa,xb=[getM(i,xn,xn) for i in 'ab']
xa,xb

⎛⎡a_{{0}{0}}  a_{{0}{1}}⎤  ⎡b_{{0}{0}}  b_{{0}{1}}⎤⎞
⎜⎢                      ⎥, ⎢                      ⎥⎟
⎝⎣a_{{1}{0}}  a_{{1}{1}}⎦  ⎣b_{{1}{0}}  b_{{1}{1}}⎦⎠

In [41]:
sp.simplify(1/xa.det()*xa.adjugate()-xa.inv())

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

In [42]:
sp.simplify(xa.inv().det()-1/xa.det())

**定理:**$|A|\not =0 \leftrightharpoons 方阵A可逆$,此时,称矩阵$A$为非奇异矩阵.

**推论:**如果$n$阶方阵$A、B$可逆,那么$A^{-1}、A^T、\lambda A(\lambda\not=0)与AB$也可逆,则
* ${(A^{-1})}^{-1}=A$,
* ${(A^T)}^{-1}={(A^{-1})}^T$,
* ${(\lambda A)}^{-1}=\frac 1 \lambda A^{-1}$,
* ${(AB)}^{-1}=B^{-1}A^{-1}$.

In [43]:
sp.simplify(xa.inv().inv()-xa)

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

In [44]:
sp.simplify(xa.T.inv()-xa.inv().T)

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

In [45]:
sp.simplify((x*xa).inv()-1/x*xa.inv())

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

In [46]:
sp.simplify((xa*xb).inv()-xb.inv()*xa.inv())

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

线性变换$\begin{cases}y_1=a_{11}x_1+\dots+a_{1n}x_n\\\vdots\\y_m=a_{m1}x_1+\dots+a_{mn}x_n \end{cases}$<br>
的系数矩阵是一个$n$阶方阵$A$,若记$X=\begin{bmatrix}x_1\\\vdots\\x_n\end{bmatrix},Y=\begin{bmatrix}y_1\\\vdots\\y_n\end{bmatrix}$<br>
则上述线性变换可记作$Y=AX$,反过来求求$变量y到x的线性变换相当于求方阵A$的逆矩阵$X=A^{-1}Y$.

# 4.矩阵分块法
由于某些条件的限制,我们经常会遇到大型文件无法上传的情况,这时候可以使用数据分批处理的方法.

用一些横线和竖线将矩阵分成若干个小块,这种操作称为**对矩阵进行分块**;<br>
其中每一个小块称为**矩阵的子块**;<br>
矩阵分块后，以子块为元素的形式上的矩阵称为**分块矩阵**.<br>

对于行数和列数较高的的矩阵,运算时采用分块法,可以使大矩阵的运算化成小矩阵的运算,体现了**化整为零**的思想.

In [47]:
xn=6
xa,xb=(getM(i,xn,xn) for i in 'ab')
xa,xb

⎛⎡a_{{0}{0}}  a_{{0}{1}}  a_{{0}{2}}  a_{{0}{3}}  a_{{0}{4}}  a_{{0}{5}}⎤  ⎡b_
⎜⎢                                                                      ⎥  ⎢  
⎜⎢a_{{1}{0}}  a_{{1}{1}}  a_{{1}{2}}  a_{{1}{3}}  a_{{1}{4}}  a_{{1}{5}}⎥  ⎢b_
⎜⎢                                                                      ⎥  ⎢  
⎜⎢a_{{2}{0}}  a_{{2}{1}}  a_{{2}{2}}  a_{{2}{3}}  a_{{2}{4}}  a_{{2}{5}}⎥  ⎢b_
⎜⎢                                                                      ⎥, ⎢  
⎜⎢a_{{3}{0}}  a_{{3}{1}}  a_{{3}{2}}  a_{{3}{3}}  a_{{3}{4}}  a_{{3}{5}}⎥  ⎢b_
⎜⎢                                                                      ⎥  ⎢  
⎜⎢a_{{4}{0}}  a_{{4}{1}}  a_{{4}{2}}  a_{{4}{3}}  a_{{4}{4}}  a_{{4}{5}}⎥  ⎢b_
⎜⎢                                                                      ⎥  ⎢  
⎝⎣a_{{5}{0}}  a_{{5}{1}}  a_{{5}{2}}  a_{{5}{3}}  a_{{5}{4}}  a_{{5}{5}}⎦  ⎣b_

{{0}{0}}  b_{{0}{1}}  b_{{0}{2}}  b_{{0}{3}}  b_{{0}{4}}  b_{{0}{5}}⎤⎞
                                                           

## 4.1.分块矩阵的加法

In [48]:
xm=xn//2
sp.Matrix.row_join(xa[:,:xm]+xb[:,:xm],xa[:,xm:]+xb[:,xm:])

⎡a_{{0}{0}} + b_{{0}{0}}  a_{{0}{1}} + b_{{0}{1}}  a_{{0}{2}} + b_{{0}{2}}  a_
⎢                                                                             
⎢a_{{1}{0}} + b_{{1}{0}}  a_{{1}{1}} + b_{{1}{1}}  a_{{1}{2}} + b_{{1}{2}}  a_
⎢                                                                             
⎢a_{{2}{0}} + b_{{2}{0}}  a_{{2}{1}} + b_{{2}{1}}  a_{{2}{2}} + b_{{2}{2}}  a_
⎢                                                                             
⎢a_{{3}{0}} + b_{{3}{0}}  a_{{3}{1}} + b_{{3}{1}}  a_{{3}{2}} + b_{{3}{2}}  a_
⎢                                                                             
⎢a_{{4}{0}} + b_{{4}{0}}  a_{{4}{1}} + b_{{4}{1}}  a_{{4}{2}} + b_{{4}{2}}  a_
⎢                                                                             
⎣a_{{5}{0}} + b_{{5}{0}}  a_{{5}{1}} + b_{{5}{1}}  a_{{5}{2}} + b_{{5}{2}}  a_

{{0}{3}} + b_{{0}{3}}  a_{{0}{4}} + b_{{0}{4}}  a_{{0}{5}} + b_{{0}{5}}⎤
                                                         

## 4.2.分块矩阵的数乘

In [49]:
sp.Matrix.row_join(xa[:,:xm]*x,xa[:,:xm]*x)

⎡a_{{0}{0}}⋅x  a_{{0}{1}}⋅x  a_{{0}{2}}⋅x  a_{{0}{0}}⋅x  a_{{0}{1}}⋅x  a_{{0}{
⎢                                                                             
⎢a_{{1}{0}}⋅x  a_{{1}{1}}⋅x  a_{{1}{2}}⋅x  a_{{1}{0}}⋅x  a_{{1}{1}}⋅x  a_{{1}{
⎢                                                                             
⎢a_{{2}{0}}⋅x  a_{{2}{1}}⋅x  a_{{2}{2}}⋅x  a_{{2}{0}}⋅x  a_{{2}{1}}⋅x  a_{{2}{
⎢                                                                             
⎢a_{{3}{0}}⋅x  a_{{3}{1}}⋅x  a_{{3}{2}}⋅x  a_{{3}{0}}⋅x  a_{{3}{1}}⋅x  a_{{3}{
⎢                                                                             
⎢a_{{4}{0}}⋅x  a_{{4}{1}}⋅x  a_{{4}{2}}⋅x  a_{{4}{0}}⋅x  a_{{4}{1}}⋅x  a_{{4}{
⎢                                                                             
⎣a_{{5}{0}}⋅x  a_{{5}{1}}⋅x  a_{{5}{2}}⋅x  a_{{5}{0}}⋅x  a_{{5}{1}}⋅x  a_{{5}{

2}}⋅x⎤
     ⎥
2}}⋅x⎥
     ⎥
2}}⋅x⎥
     ⎥
2}}⋅x⎥
     ⎥
2}}⋅x⎥
     ⎥
2}}⋅x⎦

## 4.3.分块矩阵的乘法
设$A\in\mathbb R^{m\times l},B\in\mathbb{R}^{l\times n}$

In [50]:
xn=4
xm=xn//2
xa,xb=(getM(i,xn,xn) for i in 'ab')
xa,xb

⎛⎡a_{{0}{0}}  a_{{0}{1}}  a_{{0}{2}}  a_{{0}{3}}⎤  ⎡b_{{0}{0}}  b_{{0}{1}}  b_
⎜⎢                                              ⎥  ⎢                          
⎜⎢a_{{1}{0}}  a_{{1}{1}}  a_{{1}{2}}  a_{{1}{3}}⎥  ⎢b_{{1}{0}}  b_{{1}{1}}  b_
⎜⎢                                              ⎥, ⎢                          
⎜⎢a_{{2}{0}}  a_{{2}{1}}  a_{{2}{2}}  a_{{2}{3}}⎥  ⎢b_{{2}{0}}  b_{{2}{1}}  b_
⎜⎢                                              ⎥  ⎢                          
⎝⎣a_{{3}{0}}  a_{{3}{1}}  a_{{3}{2}}  a_{{3}{3}}⎦  ⎣b_{{3}{0}}  b_{{3}{1}}  b_

{{0}{2}}  b_{{0}{3}}⎤⎞
                    ⎥⎟
{{1}{2}}  b_{{1}{3}}⎥⎟
                    ⎥⎟
{{2}{2}}  b_{{2}{3}}⎥⎟
                    ⎥⎟
{{3}{2}}  b_{{3}{3}}⎦⎠

In [51]:
xs=[sp.Matrix(((i[:xm,:xm],i[:xm,xm:]),(i[xm:,:xm],i[xm:,xm:]))) for i in [xa,xb]]
xs

⎡⎡⎡a_{{0}{0}}  a_{{0}{1}}⎤  ⎡a_{{0}{2}}  a_{{0}{3}}⎤⎤  ⎡⎡b_{{0}{0}}  b_{{0}{1}
⎢⎢⎢                      ⎥  ⎢                      ⎥⎥  ⎢⎢                     
⎢⎢⎣a_{{1}{0}}  a_{{1}{1}}⎦  ⎣a_{{1}{2}}  a_{{1}{3}}⎦⎥  ⎢⎣b_{{1}{0}}  b_{{1}{1}
⎢⎢                                                  ⎥, ⎢                      
⎢⎢⎡a_{{2}{0}}  a_{{2}{1}}⎤  ⎡a_{{2}{2}}  a_{{2}{3}}⎤⎥  ⎢⎡b_{{2}{0}}  b_{{2}{1}
⎢⎢⎢                      ⎥  ⎢                      ⎥⎥  ⎢⎢                     
⎣⎣⎣a_{{3}{0}}  a_{{3}{1}}⎦  ⎣a_{{3}{2}}  a_{{3}{3}}⎦⎦  ⎣⎣b_{{3}{0}}  b_{{3}{1}

}⎤  ⎡b_{{0}{2}}  b_{{0}{3}}⎤⎤⎤
 ⎥  ⎢                      ⎥⎥⎥
}⎦  ⎣b_{{1}{2}}  b_{{1}{3}}⎦⎥⎥
                            ⎥⎥
}⎤  ⎡b_{{2}{2}}  b_{{2}{3}}⎤⎥⎥
 ⎥  ⎢                      ⎥⎥⎥
}⎦  ⎣b_{{3}{2}}  b_{{3}{3}}⎦⎦⎦

In [52]:
xs=xs[0]*xs[1]
xs

⎡⎡a_{{0}{0}}⋅b_{{0}{0}} + a_{{0}{1}}⋅b_{{1}{0}} + a_{{0}{2}}⋅b_{{2}{0}} + a_{{
⎢⎢                                                                            
⎢⎣a_{{1}{0}}⋅b_{{0}{0}} + a_{{1}{1}}⋅b_{{1}{0}} + a_{{1}{2}}⋅b_{{2}{0}} + a_{{
⎢                                                                             
⎢⎡a_{{2}{0}}⋅b_{{0}{0}} + a_{{2}{1}}⋅b_{{1}{0}} + a_{{2}{2}}⋅b_{{2}{0}} + a_{{
⎢⎢                                                                            
⎣⎣a_{{3}{0}}⋅b_{{0}{0}} + a_{{3}{1}}⋅b_{{1}{0}} + a_{{3}{2}}⋅b_{{2}{0}} + a_{{

0}{3}}⋅b_{{3}{0}}  a_{{0}{0}}⋅b_{{0}{1}} + a_{{0}{1}}⋅b_{{1}{1}} + a_{{0}{2}}⋅
                                                                              
1}{3}}⋅b_{{3}{0}}  a_{{1}{0}}⋅b_{{0}{1}} + a_{{1}{1}}⋅b_{{1}{1}} + a_{{1}{2}}⋅
                                                                              
2}{3}}⋅b_{{3}{0}}  a_{{2}{0}}⋅b_{{0}{1}} + a_{{2}{1}}⋅b_{{1}{1}} + a_{{2}{2}}⋅
                                                   

In [53]:
xs=sp.Matrix.row_join(*[sp.Matrix.col_join(*[xs[i,j] for i in range(xm)]) for j in range(xm)])
xs

⎡a_{{0}{0}}⋅b_{{0}{0}} + a_{{0}{1}}⋅b_{{1}{0}} + a_{{0}{2}}⋅b_{{2}{0}} + a_{{0
⎢                                                                             
⎢a_{{1}{0}}⋅b_{{0}{0}} + a_{{1}{1}}⋅b_{{1}{0}} + a_{{1}{2}}⋅b_{{2}{0}} + a_{{1
⎢                                                                             
⎢a_{{2}{0}}⋅b_{{0}{0}} + a_{{2}{1}}⋅b_{{1}{0}} + a_{{2}{2}}⋅b_{{2}{0}} + a_{{2
⎢                                                                             
⎣a_{{3}{0}}⋅b_{{0}{0}} + a_{{3}{1}}⋅b_{{1}{0}} + a_{{3}{2}}⋅b_{{2}{0}} + a_{{3

}{3}}⋅b_{{3}{0}}  a_{{0}{0}}⋅b_{{0}{1}} + a_{{0}{1}}⋅b_{{1}{1}} + a_{{0}{2}}⋅b
                                                                              
}{3}}⋅b_{{3}{0}}  a_{{1}{0}}⋅b_{{0}{1}} + a_{{1}{1}}⋅b_{{1}{1}} + a_{{1}{2}}⋅b
                                                                              
}{3}}⋅b_{{3}{0}}  a_{{2}{0}}⋅b_{{0}{1}} + a_{{2}{1}}⋅b_{{1}{1}} + a_{{2}{2}}⋅b
                                                   

In [54]:
xs-xa*xb

⎡0  0  0  0⎤
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎣0  0  0  0⎦

## 4.4.分块矩阵的转置

In [55]:
xs=sp.Matrix(((xa[:xm,:xm],xa[:xm,xm:]),(xa[xm:,:xm],xa[xm:,xm:])))
xs

⎡⎡a_{{0}{0}}  a_{{0}{1}}⎤  ⎡a_{{0}{2}}  a_{{0}{3}}⎤⎤
⎢⎢                      ⎥  ⎢                      ⎥⎥
⎢⎣a_{{1}{0}}  a_{{1}{1}}⎦  ⎣a_{{1}{2}}  a_{{1}{3}}⎦⎥
⎢                                                  ⎥
⎢⎡a_{{2}{0}}  a_{{2}{1}}⎤  ⎡a_{{2}{2}}  a_{{2}{3}}⎤⎥
⎢⎢                      ⎥  ⎢                      ⎥⎥
⎣⎣a_{{3}{0}}  a_{{3}{1}}⎦  ⎣a_{{3}{2}}  a_{{3}{3}}⎦⎦

分块矩阵不仅形式上的大矩阵进行转置，而且每一个子块小矩阵也进行转置

In [56]:
xs=xs.T
xs=sp.Matrix.row_join(*[sp.Matrix.col_join(*[xs[i,j].T for i in range(xm)]) for j in range(xm)])
xs

⎡a_{{0}{0}}  a_{{1}{0}}  a_{{2}{0}}  a_{{3}{0}}⎤
⎢                                              ⎥
⎢a_{{0}{1}}  a_{{1}{1}}  a_{{2}{1}}  a_{{3}{1}}⎥
⎢                                              ⎥
⎢a_{{0}{2}}  a_{{1}{2}}  a_{{2}{2}}  a_{{3}{2}}⎥
⎢                                              ⎥
⎣a_{{0}{3}}  a_{{1}{3}}  a_{{2}{3}}  a_{{3}{3}}⎦

In [57]:
xa.T==xs

True

## 4.5.分块对角矩阵
**定义:**设$A$是$n$阶矩阵，若：
1. $A$的分块矩阵只有在对角线上有非零子块,
2. 其余子块都为零矩阵,
3. 对角线上的子块都是方阵,

那么称$A$为**分块对角矩阵**.如:

In [58]:
xa=[getM(i,j,k) for i,j,k in (('a',1,1),('b',1,1),('c',2,2))]
xs=sp.diag(*xa)
xa,xs

⎛                                              ⎡a_{0}    0        0           
⎜                                              ⎢                              
⎜⎡                  ⎡c_{{0}{0}}  c_{{0}{1}}⎤⎤  ⎢  0    b_{0}      0           
⎜⎢[a_{0}], [b_{0}], ⎢                      ⎥⎥, ⎢                              
⎜⎣                  ⎣c_{{1}{0}}  c_{{1}{1}}⎦⎦  ⎢  0      0    c_{{0}{0}}  c_{{
⎜                                              ⎢                              
⎝                                              ⎣  0      0    c_{{1}{0}}  c_{{

0     ⎤⎞
      ⎥⎟
0     ⎥⎟
      ⎥⎟
0}{1}}⎥⎟
      ⎥⎟
1}{1}}⎦⎠

**分块对角阵的性质**:
* $|A|=|A_1|\dots|A_s|$
* 若$|A_i|\not =0,则|A|\not=0$,并且矩阵$A$的逆等于分块矩阵各子块取逆之后的新矩阵。

In [59]:
reduce(lambda x,y:x+y,range(6))

In [60]:
xs.det(),reduce(lambda i,j:i*j,[i.det() for i in xa])

In [61]:
sp.simplify(xs.det()-reduce(lambda i,j:i*j,[i.det() for i in xa]))

In [62]:
xs.inv(),sp.diag(*[i.inv() for i in xa])

⎛⎡  1                                                                         
⎜⎢─────    0                          0                                       
⎜⎢a_{0}                                                                       
⎜⎢                                                                            
⎜⎢         1                                                                  
⎜⎢  0    ─────                        0                                       
⎜⎢       b_{0}                                                                
⎜⎢                                                                            
⎜⎢                                c_{{1}{1}}                                  
⎜⎢  0      0    ─────────────────────────────────────────────  ───────────────
⎜⎢              c_{{0}{0}}⋅c_{{1}{1}} - c_{{0}{1}}⋅c_{{1}{0}}  c_{{0}{0}}⋅c_{{
⎜⎢                                                                            
⎜⎢                               -c_{{1}{0}}        

In [63]:
xs.inv()==sp.diag(*[i.inv() for i in xa])

True